# Model Building

in this notebook, we will build a CNN model with transfer learning such as VGG16, ResNet50 or 101, ect. we want to reach a high accuracy something near to 99%. 

Notebook agenda:
- imporint the libraries
- EDA
- pre-processing (like resize images, data augmentation)
- model and transfer learning
- create a function for bounded box
- evaluate the model